### Configurar semestre actual

In [1]:
archivo_json = "2025-2"

### Utilidades

In [2]:
import re
from html import unescape

def extract_description(program_str: str) -> str:
    # Limpia etiquetas HTML y caracteres especiales
    clean = re.sub(r'<[^>]+>', '', program_str)
    clean = unescape(clean)

    # Unifica saltos de línea y quita espacios de más
    clean = re.sub(r'\r\n?|\n', '\n', clean)
    clean = re.sub(r'\t+', ' ', clean)

    # Busca secciones como "I. DESCRIPCIÓN", "II.DESCRIPCION", etc.
    pattern = re.compile(r"(?m)^([IVXLCDM]+)[\.\s]+(.{3,100})$")

    matches = list(pattern.finditer(clean))

    for i, match in enumerate(matches):
        title = match.group(2).upper()
        if "DESCRIPCION" in title or "DESCRIPCIÓN" in title:
            start = match.end()
            end = matches[i + 1].start() if i + 1 < len(matches) else len(clean)
            return clean[start:end].strip()

    return ""


### Unir todos los semestres en 1
Esto es para generar el `cursos-simplificado.json`, que son los cursos que alguna vez se han dado en la universidad

In [3]:

##
# SI TIENE UN NUEVO ARCHIVO JSON, DEBE SER AGREGADO AQUÍ
# ACTUALMENTE, LOS ARCHIVOS JSON SON:
# - 2025-2.json
# - 2025-1.json 
# - 2024-3.json // El 3 es TAV
# - 2024-2.json
##

import json

with open("2025-2.json", "r", encoding="utf-8") as f:
    data_2025_2 = json.load(f)
for i in data_2025_2:
    data_2025_2[i]["last_semester"] = "2025-2"

with open("2025-1.json", "r", encoding="utf-8") as f:
    data_2025_1 = json.load(f)
for i in data_2025_1:
    data_2025_1[i]["last_semester"] = "2025-1"

with open("2024-3.json", "r", encoding="utf-8") as f:
    data_2024_3 = json.load(f)
for i in data_2024_3:
    data_2024_3[i]["last_semester"] = "2024-3"

with open("2024-2.json", "r", encoding="utf-8") as f:
    data_2024_2 = json.load(f)
for i in data_2024_2:
    data_2024_2[i]["last_semester"] = "2024-2"

# Combinamos: 2024-2 primero, y luego sobrescribimos con 2025-1
combined_data = data_2024_2.copy()
combined_data.update(data_2024_3)
combined_data.update(data_2025_1)  # sobrescribe duplicados con los de 2025-1
combined_data.update(data_2025_2)  # sobrescribe duplicados con los de 2025-2

In [4]:


simplified_data = {}

for code, course in combined_data.items():
    sections = course.get("sections", {})
    
    # Agrupar valores únicos de secciones
    formats = set()
    campuses = set()
    is_removable = set()
    is_special = set()
    is_english = set()
    categories = set()

    for section in sections.values():
        formats.add(section.get("format"))
        campuses.add(section.get("campus"))
        is_removable.add(section.get("is_removable"))
        is_special.add(section.get("is_special"))
        is_english.add(section.get("is_english"))
        categories.add(section.get("category", ""))
    
    # Extraer descripción del programa
    program_raw = course.get("program", "")
    description = extract_description(program_raw)

    simplified_data[code] = {
        "sigle": code,
        "name": course["name"],
        "credits": course["credits"],
        "req": course["req"],
        "conn": course["conn"],
        "restr": course["restr"],
        "equiv": course["equiv"],
        "school": course["school"],
        "area": course.get("area", ""),
        "categories": list(categories),
        "format": list(formats),
        "campus": list(campuses),
        "is_removable": list(is_removable),
        "is_special": list(is_special),
        "is_english": list(is_english),
        "description": description,
        "last_semester": course["last_semester"]
    }

    if description == "" and program_raw != "Programa de curso no disponible":
        print(f"Advertencia: No se encontró descripción para el curso {code}")

simplified_data["OSUC500"] = {
    "sigle": "OSUC500",
    "name": "Busca Ramos Feedback",
    "credits": -10,
    "req": "",
    "conn": "",
    "restr": "",
    "equiv": "",
    "school": "OSUC",
    "area": "",
    "category": "",
    "format": ["Presencial"],
    "campus": ["San Joaquín", "Oriente", "Villarrica", "Casa Central", "Lo Contador"],
    "is_removable": [False],
    "is_special": [False],
    "is_english": [False],
    "description": "¿Tienes comentarios o sugerencias? Este es tu espacio para compartir críticas y ayudarnos a mejorar el contenido de Busca Ramos. ¡Tu opinión cuenta!",
    "last_semester": archivo_json,
    "categories": ["Feedback", "Mejora Continua"]
}
# Guardar resultado
with open("cursos-simplificado.json", "w", encoding="utf-8") as f:
    json.dump(simplified_data, f, ensure_ascii=False, indent=2)


Advertencia: No se encontró descripción para el curso ACO4001
Advertencia: No se encontró descripción para el curso ACO4002
Advertencia: No se encontró descripción para el curso AGP4020
Advertencia: No se encontró descripción para el curso ARO105D
Advertencia: No se encontró descripción para el curso AST0112
Advertencia: No se encontró descripción para el curso ASP5406
Advertencia: No se encontró descripción para el curso BIO231C
Advertencia: No se encontró descripción para el curso COM024
Advertencia: No se encontró descripción para el curso COM601
Advertencia: No se encontró descripción para el curso DDE3147
Advertencia: No se encontró descripción para el curso DER501S
Advertencia: No se encontró descripción para el curso DER503G
Advertencia: No se encontró descripción para el curso DMD3925
Advertencia: No se encontró descripción para el curso EAA2095
Advertencia: No se encontró descripción para el curso EAA3230
Advertencia: No se encontró descripción para el curso EAE4105
Advertenci

In [5]:
len(combined_data)

7323

# SQL parser
Pasar los datos a un **.sql** para poder subirlos a bdd (Busca Ramos formato)

In [6]:
import os

# Generar sentencias SQL para course_summary
sql_statements = []

for course in simplified_data.values():
    sigle = course["sigle"]

    columns = [
        "sigle",
        "likes",
        "superlikes",
        "dislikes",
        "votes_low_workload",
        "votes_medium_workload",
        "votes_high_workload",
        "votes_mandatory_attendance",
        "votes_optional_attendance",
        "avg_weekly_hours",
        "sort_index"
    ]

    # Valores iniciales por defecto
    values = [
        f"'{sigle}'",
        "0", "0", "0",         # likes, superlikes, dislikes
        "0", "0", "0",         # workload votes
        "0", "0",              # attendance votes
        "0.0",                 # avg_weekly_hours
        "0"                   # sort_index
    ]

    updates = [f"{col}=EXCLUDED.{col}" for col in columns if col != "sigle"]

    sql = (
        f"INSERT INTO course_summary ({', '.join(columns)})\n"
        f"VALUES ({', '.join(values)})\n"
        f"ON CONFLICT (sigle) DO NOTHING;"
    )

    sql_statements.append(sql)

# Crear carpeta de salida
os.makedirs("sql_output", exist_ok=True)

# Escribir en archivos de 100 sentencias cada uno
for i in range(0, len(sql_statements), 100):
    chunk = sql_statements[i:i + 100]
    file_index = (i // 100) + 1
    with open(f"sql_output/{file_index}.sql", "w", encoding="utf-8") as f:
        f.write("\n\n".join(chunk))

print(f"✅ {((len(sql_statements) - 1) // 100) + 1} archivos SQL generados en carpeta 'sql_output/' para 'course_summary'")


✅ 74 archivos SQL generados en carpeta 'sql_output/' para 'course_summary'


### Obtener la informacion de **solo las secciones del semestre configurado**

In [7]:
import json

# Cargar datos desde el archivo JSON
with open(f"{archivo_json}.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Escribir archivo NDJSON con solo 'sigle', 'sections' y 'name'
with open(f"courses-sections.ndjson", "w", encoding="utf-8") as f:
    for curso in data.values():
        ndjson_obj = {
            "sigle": curso.get("sigle"),
            "sections": curso.get("sections"),
            "name": curso.get("name"),
        }
        f.write(json.dumps(ndjson_obj, ensure_ascii=False) + "\n")
